# Enrich a CSV with data from an API

Data sources:

- The [Goodreads books](https://www.kaggle.com/jealousleopard/goodreadsbooks) dataset from Kaggle.
- The [Open Library Books API](https://openlibrary.org/dev/docs/api/books)

The goal is to enrich the Kaggle dataset with additional information from the Open Library Books API 

- The cover URL of the book
- List of subjects (Science, Humor, Travel, etc.)

This notebook is adapted from the Le Wagon Data Science Bootcamp challenge

In [1]:
# imports
import requests
import numpy as np
import pandas as pd

In [2]:
# load csv from Kaggle
books_df = pd.read_csv('books.csv', on_bad_lines='skip')
books_df.head(10)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic
5,9,"Unauthorized Harry Potter Book Seven News: ""Ha...",W. Frederick Zimmerman,3.74,0976540606,9780976540601,en-US,152,19,1,4/26/2005,Nimble Books
6,10,Harry Potter Collection (Harry Potter #1-6),J.K. Rowling,4.73,0439827604,9780439827607,eng,3342,28242,808,9/12/2005,Scholastic
7,12,The Ultimate Hitchhiker's Guide: Five Complete...,Douglas Adams,4.38,0517226952,9780517226957,eng,815,3628,254,11/1/2005,Gramercy Books
8,13,The Ultimate Hitchhiker's Guide to the Galaxy ...,Douglas Adams,4.38,0345453743,9780345453747,eng,815,249558,4080,4/30/2002,Del Rey Books
9,14,The Hitchhiker's Guide to the Galaxy (Hitchhik...,Douglas Adams,4.22,1400052920,9781400052929,eng,215,4930,460,8/3/2004,Crown


In [3]:
books_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11123 entries, 0 to 11122
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   bookID              11123 non-null  int64  
 1   title               11123 non-null  object 
 2   authors             11123 non-null  object 
 3   average_rating      11123 non-null  float64
 4   isbn                11123 non-null  object 
 5   isbn13              11123 non-null  int64  
 6   language_code       11123 non-null  object 
 7     num_pages         11123 non-null  int64  
 8   ratings_count       11123 non-null  int64  
 9   text_reviews_count  11123 non-null  int64  
 10  publication_date    11123 non-null  object 
 11  publisher           11123 non-null  object 
dtypes: float64(1), int64(5), object(6)
memory usage: 1.0+ MB


## Add cover URL from API

In [4]:
def fetch_cover(isbn):
    url = f'http://openlibrary.org/api/books?bibkeys=ISBN:{isbn}&jscmd=data&format=json'
    response = requests.get(url)
    if response.status_code != 200:
        return ''
    data = response.json()
    if f"ISBN:{isbn}" in data:
        if 'cover' in data[f'ISBN:{isbn}'].keys():
            return data[f'ISBN:{isbn}']['cover']['small']
        return 'No cover URL available'
    return ''

In [5]:
# test cover
print(fetch_cover('0439785960'))
print(fetch_cover('0345453743'))

https://covers.openlibrary.org/b/id/9326654-S.jpg
No cover URL available


In [6]:
# create new column
books_df['Cover URL']=''
books_df.head(1)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,Cover URL
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.,


In [ ]:
%%time
# Fetch cover URL for first 200 books in the DF
for index, row in books_df.sample(200).iterrows():
    print(f"Fetching Cover URL for {row['authors']} - {row['title']}")
    cover = fetch_cover(row['isbn'])
    books_df.loc[index, 'Cover URL'] = cover

Fetching Cover URL for Deborah Blum - Ghost Hunters: William James and the Search for Scientific Proof of Life After Death
Fetching Cover URL for Orson Scott Card/Piotr W. Cholewa - Xenocide (Ender's Saga  #3)
Fetching Cover URL for L.M. Montgomery/Susan O'Malley - Anne of Avonlea
Fetching Cover URL for Georg Wilhelm Friedrich Hegel - The Essential Writings
Fetching Cover URL for Hideyuki Kikuchi/Yoshitaka Amano - Vampire Hunter D Volume 07: Mysterious Journey to the North Sea - Part One
Fetching Cover URL for William J. Bennett - From the Age of Discovery to a World at War (America: The Last Best Hope #1)
Fetching Cover URL for Richard M. Weaver - Ideas Have Consequences
Fetching Cover URL for Junichi Satō/Kaori Naruse - Prétear  Vol. 4 (Prétear  #4)
Fetching Cover URL for Robin McKinley - Spindle's End
Fetching Cover URL for Luigi Pirandello/Henry Reed - Collected Plays: Henry IV  The Man with the Flower in His Mouth  Right You Are
Fetching Cover URL for Tess Gerritsen - Vanish (Rizz

Fetching Cover URL for Mary Wollstonecraft Shelley/Susan J. Wolfson - Frankenstein
Fetching Cover URL for David Morrell - The Totem
Fetching Cover URL for Vijay Prashad - Everybody Was Kung Fu Fighting: Afro-Asian Connections and the Myth of Cultural Purity
Fetching Cover URL for Amy Tan - The Joy Luck Club
Fetching Cover URL for Jean Sasson - Princess Sultana's Daughters
Fetching Cover URL for Iain M. Banks - The Algebraist
Fetching Cover URL for George Eliot/Thomas A. Noble - Scenes of Clerical Life
Fetching Cover URL for P.C. Cast - Divine By Choice (Partholon  #2)
Fetching Cover URL for Anaïs Nin - Henry and June: From "A Journal of Love": The Unexpurgated Diary of Anaïs Nin  1931-1932
Fetching Cover URL for Toni Morrison - The Bluest Eye
Fetching Cover URL for Mike Carey/Glenn Fabry/Neil Gaiman - Neil Gaiman's Neverwhere
Fetching Cover URL for L.M. Montgomery - The Road to Yesterday (Anne of Green Gables  #9)
Fetching Cover URL for José Saramago/Giovanni Pontiero - Blindness
Fetch

In [ ]:
# Check the DF
books_df.head(10)

## Add subject name for API

In [ ]:
def fetch_subject(isbn):
    url = f'http://openlibrary.org/api/books?bibkeys=ISBN:{isbn}&jscmd=data&format=json'
    response = requests.get(url)
    if response.status_code != 200:
        return ''
    data = response.json()
    if f"ISBN:{isbn}" in data:
        if 'subjects' in data[f'ISBN:{isbn}'].keys():
            return data[f'ISBN:{isbn}']['subjects'][0]['name']
        return 'No subject name available'
    return ''

In [ ]:
# test function
fetch_subject('1400052920')

In [ ]:
# create new column
books_df['Subject']=''
books_df.head(1)

In [ ]:
%%time
# Fetch subject for first 200 books in the DF
for index, row in books_df.sample(200).iterrows():
    print(f"Fetching subject for {row['authors']} - {row['title']}")
    subject = fetch_subject(row['isbn'])
    books_df.loc[index, 'Subject'] = subject

In [ ]:
# Check the DF
books_df.head(10)